In [1]:
import time
import os
import numpy as np
import pandas as pd
import nibabel as nib
from scipy import ndimage
from skimage.measure import label, regionprops
from scipy.ndimage.filters import median_filter
from scipy.ndimage.measurements import label
import pickle

In [2]:
from src import Graphs
from src import QAP
from src import utils
from src import metrics
from src import refinement as ref
import pipeline_QAP

In [3]:
def get_largest_cc_results(image_cnn, nb_classes):

    image = np.zeros(image_cnn.shape)

    for i in range(1, nb_classes+1):
        mask = np.where(image_cnn == i, 1, 0).astype(np.bool)
        lbl = label(mask, connectivity=2)
        regions = regionprops(lbl)
        biggest_region = None
        for region in regions:
            if biggest_region is None:
                biggest_region = region
            else:
                if biggest_region.area < region.area:
                    biggest_region = region
        image = np.where(lbl == biggest_region.label, i, image)

    return image

In [4]:
def save_img_to_nii(img, path):

    nib_image = nib.Nifti1Image(img, np.eye(4))

    nib_image.header.get_xyzt_units()
    nib_image.to_filename(path)

In [5]:
def load_image_from_dict(image_lbl_path, dict_path, nb_classes):
    loaded_image = nib.load(image_lbl_path).get_fdata()
    with open(dict_path, 'rb') as handle:
        b = pickle.load(handle)

    dims = list(loaded_image.shape)

    dims.append(nb_classes+1)

    base_image = np.zeros(dims)

    uni = np.unique(loaded_image)

    for value in uni:

        region_mask = np.where(loaded_image == value, 1, 0)

        if len(loaded_image.shape) > 2:
            proba_mask = np.expand_dims(region_mask, axis=3) * b[value]
        else:
            proba_mask = np.expand_dims(region_mask, axis=2) * b[value]

        base_image = base_image + proba_mask
    return base_image

In [6]:
percentage = 50

iteration = 1

KNOWLEDGES_PATH = os.path.join(".", "sub_datasets_brain", str(percentage), str(iteration),  "knowledges", "Brain_Dist_relative.npy")

DICT_DIRECTORY = os.path.join(".", "sub_datasets_brain", str(percentage), str(iteration), "dict")

RESULTS_DIRECTORY = os.path.join(".", "sub_datasets_brain", str(percentage), str(iteration), "results")


In [7]:
alpha = 0.5

nb_classes = 14

specifier = "centroid"

max_node_matching = 2
max_node_refinement = np.inf

In [8]:
if os.path.isdir(RESULTS_DIRECTORY) == False:
    os.mkdir(RESULTS_DIRECTORY)

RESULTS_DIRECTORY_specifier = os.path.join(".", "sub_datasets_brain", str(percentage), str(iteration), "results", specifier)

if os.path.isdir(RESULTS_DIRECTORY_specifier) == False:
    os.mkdir(RESULTS_DIRECTORY_specifier)

In [9]:
nb_files = int(len(os.listdir(DICT_DIRECTORY)) / 2)

times = []
errors = []

In [10]:
for k in range(0, nb_files):

    t_initial = time.time()

    try:
        patient_path = os.path.join(RESULTS_DIRECTORY_specifier, "P" + str(k))
        if os.path.isdir(patient_path) == False:
            os.mkdir(patient_path)

        """""""""""""""""""""""""""""""""""""""
            Chargement des données
        """""""""""""""""""""""""""""""""""""""

        image_lbl_path = os.path.join(DICT_DIRECTORY, str(k) + "_lbl.nii.gz")
        dict_path = os.path.join(DICT_DIRECTORY, str(k) + "_dict.pickle")
        print("ok1")
        pr_mask = load_image_from_dict(image_lbl_path, dict_path, nb_classes)

        print("ok")

        #pr_mask = np.load(os.path.join(SEGMENTATION_DIRECTORY, str(k) + "pr.npy"))
        #gt_mask = np.load(os.path.join(SEGMENTATION_DIRECTORY, str(k) + "gt.npy"))
        image_cnn = np.argmax(pr_mask, axis=len(pr_mask.shape) - 1)
        #intensity_input = np.load(os.path.join(SEGMENTATION_DIRECTORY, str(k) + "intensity.npy"))

        dims = len(image_cnn.shape)

        """""""""""""""""""""""""""""""""""""""
            Etapes de pre-traitements
        """""""""""""""""""""""""""""""""""""""

        #pr_mask = np.transpose(pr_mask, (1,2,3,0))
        
        #image_cnn = np.where(intensity_input > 0, image_cnn, 0)

        #image_cnn = ndimage.median_filter(image_cnn, 3)

        pr_mask = pr_mask[:,:,:, 1:nb_classes + 1]

        Am= Graphs.load_structural_model(KNOWLEDGES_PATH, nb_classes)

        Am = np.transpose(Am, (2,0,1))

        #Am = np.transpose(Am, (1,0,2))

        labelled_image, regions, matching_inter, merging_final = pipeline_QAP.pipeline_QAP(specifier, alpha, pr_mask, image_cnn, nb_classes, Am, max_node_matching, max_node_refinement)
        
        merging_final_filtre, distance_recap = ref.correct_merging_distance(labelled_image, regions, matching_inter, merging_final, 20)

        # Etape 4 : Obtention des images

        biggest_CC_image = get_largest_cc_results(image_cnn, nb_classes)

        matching_image = ref.create_images_from_ids(labelled_image, matching_inter, regions)

        proposal_image = ref.create_images_from_ids(labelled_image, merging_final, regions)

        post_refinement_image_CC = get_largest_cc_results(proposal_image, nb_classes)

        image_refinement_distance = ref.create_images_from_ids(labelled_image, merging_final_filtre, regions)

        #image_refinement_distance = median_filter(image_refinement_distance, 3)

        # Etape 4 : Sauvegardes des images

        cnn_output_path = os.path.join(patient_path, str(k) + "_cnn_output.nii.gz")
        save_img_to_nii(image_cnn, cnn_output_path)

        gt_path = os.path.join(patient_path, str(k) + "_gt.nii.gz")
        save_img_to_nii(gt_mask, gt_path)

        cnn_CC = os.path.join(patient_path, str(k) + "_cnn_output_CC.nii.gz")
        save_img_to_nii(biggest_CC_image, cnn_CC)
        
        proposal_path = os.path.join(patient_path, str(k) + "_proposal.nii.gz")
        save_img_to_nii(proposal_image, proposal_path)

        post_processing_path = os.path.join(patient_path, str(k) + "_proposal_CC.nii.gz")
        save_img_to_nii(post_refinement_image_CC, post_processing_path)

        """""""""""""""""""""""""""""""""""""""
                    Metrics
        """""""""""""""""""""""""""""""""""""""

        datas = ["dice", "precision", "recall", "hausdorff", "nb_CC"]

        score_cnn = metrics.all_informations_data_frame(datas, image_cnn, gt_mask)

        score_cnn_CC = metrics.all_informations_data_frame(datas, biggest_CC_image, gt_mask)

        score_before_refinement = metrics.all_informations_data_frame(datas, matching_image, gt_mask)

        score_after_refinement = metrics.all_informations_data_frame(datas, proposal_image, gt_mask)

        score_post_refinement_CC = metrics.all_informations_data_frame(datas, post_refinement_image_CC, gt_mask)

        result = pd.concat([score_cnn, score_cnn_CC, score_before_refinement, score_after_refinement, score_post_refinement_CC], axis=1, join="inner")

        print(result)

        result_path = os.path.join(patient_path, "result.csv")
        result.to_csv(result_path)

        processing_time = time.time() - t_initial
        times.append(processing_time)

    except:
        print("error on file" + str(k))
        errors.append(patient_path)

ok1


In [ ]:
with open(os.path.join(RESULTS_DIRECTORY_specifier, "error_log.txt"), "w") as output:
    output.write(str(errors))

times = np.asarray(times)
np.savetxt(os.path.join(RESULTS_DIRECTORY_specifier, "processing_time.csv"), times)